In [1]:
import numpy as np
import h5py
import time
import pandas as pd
import matplotlib.pyplot as plt
import glob
import scipy
import pickle as pkl
import matplotlib as mpl
from astropy.coordinates import SkyCoord
from astropy import units as u
import healpy as hp
import scipy
from scipy.optimize import differential_evolution
from matplotlib.colors import LogNorm
outpath = '/project/chihway/raulteixeira/data/'
n_deep, n_wide = 64, 32
N_deep, N_wide = n_deep**2, n_wide**2
nbins = 50
zmin, zmax = 0., 2.
deltaz = (zmax-zmin)/nbins
assert ((zmin>=0) & (zmax>=zmin) & (nbins>0) & (n_deep>0) & (n_wide>0))
z_edges = np.linspace(zmin,zmax,nbins) #np.arange(zmin,zmax,deltaz)
zs = .5*np.array([z_edges[i]+z_edges[i+1] for i in range(nbins-1)])
bins_0 = [.36, .63, .86]

In [2]:
### BALROG

balrog = pd.read_hdf('%sBalrogOfTheDECADE_20231216_wide_cells.hdf5'%outpath, key='hdf')
balrog.columns

Index(['ID', 'id', 'tilename', 'true_ra', 'true_dec', 'flux_r', 'flux_i',
       'flux_z', 'flux_err_r', 'flux_err_i', 'flux_err_z', 'Gold_Mask',
       'FG_Mask', 'Cont_Mask', 'detected', 'wide_cells'],
      dtype='object')

In [3]:
len(balrog)

10217154

In [4]:
balrog_clean = balrog[balrog['Cont_Mask']&balrog["FG_Mask"]]

In [5]:
balrog_clean

,ID,id,tilename,true_ra,true_dec,flux_r,flux_i,flux_z,flux_err_r,flux_err_i,flux_err_z,Gold_Mask,FG_Mask,Cont_Mask,detected,wide_cells
264963,698419852,8978,b'DES0726+2834',112.125145,28.319452,3097.178789,5683.205652,7920.476107,33.814217,46.543888,79.219168,False,True,True,1,588
264964,692279852,0,b'DES0726+2834',112.125198,28.325871,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True,1,0
264965,712517969,0,b'DES0726+2834',112.125237,28.332273,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True,1,0
264966,713086105,-99,b'DES0726+2834',112.125214,28.338707,-108.105869,-105.59732,-103.990154,-108.105869,-105.59732,-103.990154,False,True,True,0,0
264967,712734329,0,b'DES0726+2834',112.125221,28.345119,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9877028,712385835,-99,b'DES1636+1000',248.832413,10.188523,-112.941024,-109.039806,-106.564434,-112.941024,-109.039806,-106.564434,False,True,True,0,0
9877034,699700931,-99,b'DES1636+1000',248.832352,10.227047,-113.103756,-109.154974,-106.650219,-113.103756,-109.154974,-106.650219,False,True,True,0,0
9877035,699629712,-99,b'DES1636+1000',248.832367,10.233464,-113.143737,-109.183262,-106.671287,-113.143737,-109.183262,-106.671287,False,True,True,0,0
9877036,691937308,52126,b'DES1636+1000',248.832336,10.239907,595.912958,1270.319457,1485.400619,33.830359,50.799081,74.741153,True,True,True,1,805


In [6]:
deep = pd.read_hdf('%s/DES_DF_baldet_121923_64x64_cells.hdf'%outpath, key='df')
redshift_DF = pd.read_csv('/project/chihway/raulteixeira/data/deepfields_with_redshifts.csv.gz')
deep = deep.merge(redshift_DF[['ID', 'Z']], on='ID', how='left')

In [7]:
redshift_DF

,ID,RA,DEC,KNN_CLASS,BDF_FLUX_DERED_CALIB_U,BDF_FLUX_DERED_CALIB_G,BDF_FLUX_DERED_CALIB_R,BDF_FLUX_DERED_CALIB_I,BDF_FLUX_DERED_CALIB_Z,BDF_FLUX_DERED_CALIB_J,...,BDF_FLUX_ERR_DERED_CALIB_G,BDF_FLUX_ERR_DERED_CALIB_R,BDF_FLUX_ERR_DERED_CALIB_I,BDF_FLUX_ERR_DERED_CALIB_Z,BDF_FLUX_ERR_DERED_CALIB_J,BDF_FLUX_ERR_DERED_CALIB_H,BDF_FLUX_ERR_DERED_CALIB_KS,FIELD,Z,SOURCE
0,692581195,52.901547,-28.866868,1.0,78.873091,90.598396,150.538655,299.158525,604.705644,1332.163968,...,4.354539,6.915013,10.472420,22.481337,32.829235,64.018955,46.272910,C3,NaN,NaN
1,692581196,52.900143,-28.866868,1.0,427.218832,428.159639,533.924158,623.754924,930.088708,1155.277234,...,5.213569,7.697295,10.465219,21.190446,32.708205,76.518803,64.656877,C3,NaN,NaN
2,692581197,52.919675,-28.867296,1.0,204.186270,259.924072,640.058928,923.667161,990.715877,1470.225204,...,6.298630,11.451556,16.559125,29.037093,38.015369,84.564620,56.049189,C3,NaN,NaN
3,692581244,52.775273,-28.867263,1.0,30.073269,29.574921,83.823448,123.895966,123.353058,341.582272,...,4.314149,7.481621,10.854203,18.845930,82.181313,77.463570,197.384483,C3,NaN,NaN
4,692581245,52.806489,-28.867570,1.0,482.933499,471.033683,666.071153,1136.945773,1345.197659,1742.870626,...,7.675500,11.668144,18.085892,31.539036,42.248013,65.293374,58.042663,C3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727466,698430348,36.792257,-4.959126,1.0,36.794575,36.615140,113.136962,199.582800,329.359765,697.143336,...,7.078695,11.098560,19.715216,35.995655,36.685951,59.235518,85.550137,X3,NaN,NaN
727467,698430349,36.936774,-4.958946,1.0,68.959445,65.505742,173.253276,213.351412,247.441472,282.943939,...,4.587675,6.745786,10.562732,17.705044,9.978685,13.538666,17.829521,X3,NaN,NaN
727468,698430445,36.858459,-4.959729,1.0,171.296154,420.164719,816.936807,1086.845179,1158.868204,1451.461407,...,4.571781,6.509264,10.744635,17.281711,13.464829,19.914090,25.951068,X3,NaN,NaN
727469,698430493,36.867956,-4.961262,1.0,62.842978,138.970283,445.022129,1069.344794,1567.993989,3753.733436,...,4.798131,7.051887,13.929102,23.725032,17.626481,27.185082,38.591028,X3,NaN,NaN


In [8]:
#deep.to_hdf('%s/DES_DF_baldet_121923_64x64_cells_with_redshifts.hdf'%outpath, key='df')

In [9]:
deep[np.isfinite(deep['Z'])].to_hdf('%s/DES_DF_baldet_121923_64x64_cells_only_objects_with_redshifts.hdf'%outpath, key='df')

In [10]:
deep.columns

Index(['ID', 'RA', 'DEC', 'KNN_CLASS', 'FLAGS', 'FLAGSTR', 'FLAGSTR_NIR',
       'FLAGS_NIR', 'MASK_FLAGS', 'MASK_FLAGS_NIR', 'BDF_FLUX_DERED_CALIB_U',
       'BDF_FLUX_DERED_CALIB_G', 'BDF_FLUX_DERED_CALIB_R',
       'BDF_FLUX_DERED_CALIB_I', 'BDF_FLUX_DERED_CALIB_Z',
       'BDF_FLUX_DERED_CALIB_J', 'BDF_FLUX_DERED_CALIB_H',
       'BDF_FLUX_DERED_CALIB_KS', 'BDF_FLUX_ERR_DERED_CALIB_U',
       'BDF_FLUX_ERR_DERED_CALIB_G', 'BDF_FLUX_ERR_DERED_CALIB_R',
       'BDF_FLUX_ERR_DERED_CALIB_I', 'BDF_FLUX_ERR_DERED_CALIB_Z',
       'BDF_FLUX_ERR_DERED_CALIB_J', 'BDF_FLUX_ERR_DERED_CALIB_H',
       'BDF_FLUX_ERR_DERED_CALIB_KS', 'deep_cells', 'Z'],
      dtype='object')

In [11]:
balrog_deep = balrog_clean.merge(deep, on='ID', how='left') # merge balrog w/ deep catalog for later

In [12]:
balrog_deep['Gold_Mask'] = balrog_deep['Gold_Mask'].astype(bool)

In [13]:
balrog_deep = balrog_deep[np.isfinite(balrog_deep['deep_cells'])]

In [14]:
balrog_deep['true_id']=balrog_deep['ID']
balrog_deep['cell_deep']=balrog_deep['deep_cells']
balrog_deep['cell_wide_unsheared']=balrog_deep['wide_cells']

In [15]:
balrog_deep

,ID,id,tilename,true_ra,true_dec,flux_r,flux_i,flux_z,flux_err_r,flux_err_i,...,BDF_FLUX_ERR_DERED_CALIB_I,BDF_FLUX_ERR_DERED_CALIB_Z,BDF_FLUX_ERR_DERED_CALIB_J,BDF_FLUX_ERR_DERED_CALIB_H,BDF_FLUX_ERR_DERED_CALIB_KS,deep_cells,Z,true_id,cell_deep,cell_wide_unsheared
2,712517969,0,b'DES0726+2834',112.125237,28.332273,0.0,0.0,0.0,0.0,0.0,...,13.299434,20.572710,53.196909,85.623638,128.902022,1363.0,0.539000,712517969,1363.0,0
4,712734329,0,b'DES0726+2834',112.125221,28.345119,0.0,0.0,0.0,0.0,0.0,...,25.499032,39.803547,126.448011,140.112234,144.186313,1924.0,0.729000,712734329,1924.0,0
5,712633934,-99,b'DES0726+2834',112.125313,28.351561,-108.105869,-105.59732,-103.990154,-108.105869,-105.59732,...,10.951560,15.945680,4.824975,6.057633,8.417893,2303.0,0.719308,712633934,2303.0,0
7,712506387,12308,b'DES0726+2834',112.125336,28.36438,1276.740754,2015.820769,2798.824783,47.165365,63.889565,...,17.148212,27.117068,19.178142,24.150156,31.609725,2848.0,0.737000,712506387,2848.0,337
9,699329755,-99,b'DES0726+2834',112.125351,28.377209,-107.862952,-105.4233,-103.859497,-107.862952,-105.4233,...,13.199811,21.724141,34.704367,29.615324,39.350809,2788.0,NaN,699329755,2788.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6677117,699639891,46390,b'DES1636+1000',248.832443,10.169336,20809.838678,31209.550514,40426.463886,73.292362,91.119454,...,72.026525,104.026129,87.984828,92.969705,127.406914,936.0,0.228830,699639891,936.0,629
6677118,692161566,-99,b'DES1636+1000',248.832382,10.175707,-112.969921,-109.06026,-106.579671,-112.969921,-109.06026,...,19.215336,30.620487,22.178146,33.705372,45.224762,1017.0,0.249112,692161566,1017.0,0
6677119,712385835,-99,b'DES1636+1000',248.832413,10.188523,-112.941024,-109.039806,-106.564434,-112.941024,-109.039806,...,11.063636,17.972461,5.302199,7.153220,9.417211,1980.0,0.297766,712385835,1980.0,0
6677120,699700931,-99,b'DES1636+1000',248.832352,10.227047,-113.103756,-109.154974,-106.650219,-113.103756,-109.154974,...,15.410418,25.904481,32.654313,26.896936,33.094970,2839.0,NaN,699700931,2839.0,0


In [16]:
unique_ids, index, rev_index, counts_true =\
np.unique(balrog_deep.ID.values, return_counts=True, return_index=True, return_inverse=True)

InjDF = pd.DataFrame({'true_ra': balrog_deep.true_ra.values,
'true_dec': balrog_deep.true_dec.values,
'injection_counts':counts_true[rev_index],
'overlap_weight':1/counts_true[rev_index]})   

In [17]:
InjDF

,true_ra,true_dec,injection_counts,overlap_weight
0,112.125237,28.332273,19,0.052632
1,112.125221,28.345119,32,0.031250
2,112.125313,28.351561,31,0.032258
3,112.125336,28.36438,46,0.021739
4,112.125351,28.377209,22,0.045455
...,...,...,...,...
3380473,248.832443,10.169336,36,0.027778
3380474,248.832382,10.175707,30,0.033333
3380475,248.832413,10.188523,19,0.052632
3380476,248.832352,10.227047,13,0.076923


In [28]:
relevant_cols = ['true_id', 'id', 'tilename', 'true_ra', 'true_dec', 'Z', 'cell_deep', 'cell_wide_unsheared']


In [29]:
balrog_deep[relevant_cols]

,true_id,id,tilename,true_ra,true_dec,Z,cell_deep,cell_wide_unsheared
2,712517969,0,b'DES0726+2834',112.125237,28.332273,0.539000,1363.0,0
4,712734329,0,b'DES0726+2834',112.125221,28.345119,0.729000,1924.0,0
5,712633934,-99,b'DES0726+2834',112.125313,28.351561,0.719308,2303.0,0
7,712506387,12308,b'DES0726+2834',112.125336,28.36438,0.737000,2848.0,337
9,699329755,-99,b'DES0726+2834',112.125351,28.377209,NaN,2788.0,0
...,...,...,...,...,...,...,...,...
6677117,699639891,46390,b'DES1636+1000',248.832443,10.169336,0.228830,936.0,629
6677118,692161566,-99,b'DES1636+1000',248.832382,10.175707,0.249112,1017.0,0
6677119,712385835,-99,b'DES1636+1000',248.832413,10.188523,0.297766,1980.0,0
6677120,699700931,-99,b'DES1636+1000',248.832352,10.227047,NaN,2839.0,0


In [30]:
balrog_deep_2Bsaved = InjDF.merge(balrog_deep[relevant_cols], on=['true_ra', 'true_dec'], validate='1:1', how='right')

In [31]:
balrog_deep['ID']

2          712517969
4          712734329
5          712633934
7          712506387
9          699329755
             ...    
6677117    699639891
6677118    692161566
6677119    712385835
6677120    699700931
6677122    691937308
Name: ID, Length: 3380478, dtype: object

In [32]:
balrog_deep_2Bsaved

,true_ra,true_dec,injection_counts,overlap_weight,true_id,id,tilename,Z,cell_deep,cell_wide_unsheared
0,112.125237,28.332273,19,0.052632,712517969,0,b'DES0726+2834',0.539000,1363.0,0
1,112.125221,28.345119,32,0.031250,712734329,0,b'DES0726+2834',0.729000,1924.0,0
2,112.125313,28.351561,31,0.032258,712633934,-99,b'DES0726+2834',0.719308,2303.0,0
3,112.125336,28.36438,46,0.021739,712506387,12308,b'DES0726+2834',0.737000,2848.0,337
4,112.125351,28.377209,22,0.045455,699329755,-99,b'DES0726+2834',NaN,2788.0,0
...,...,...,...,...,...,...,...,...,...,...
3380473,248.832443,10.169336,36,0.027778,699639891,46390,b'DES1636+1000',0.228830,936.0,629
3380474,248.832382,10.175707,30,0.033333,692161566,-99,b'DES1636+1000',0.249112,1017.0,0
3380475,248.832413,10.188523,19,0.052632,712385835,-99,b'DES1636+1000',0.297766,1980.0,0
3380476,248.832352,10.227047,13,0.076923,699700931,-99,b'DES1636+1000',NaN,2839.0,0


In [33]:
balrog_deep_2Bsaved.to_hdf('%s/DES_DF_baldet_121923_64x64_cells_with_redshifts_colnames4uncertainties_corrected.hdf'%outpath, key='df')

/tmp/jobs/33065071/ipykernel_221976/1405431382.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['true_ra', 'true_dec', 'true_id', 'id', 'tilename'], dtype='object')]

  balrog_deep_2Bsaved.to_hdf('%s/DES_DF_baldet_121923_64x64_cells_with_redshifts_colnames4uncertainties_corrected.hdf'%outpath, key='df')
